# Directly Removing the Power Law Before Making the KDE
Now that a decent way to ignore the power law has been found and used in both `Bootstrap-Own-Data.ipynb` and `Emperical-Errors.ipynb`, let's test to see what it does when instead we try to remove the power law much earlier. This will be done for both the bootstrapping method and also the emperical errors method